Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertModel, BertTokenizer, DebertaV2Tokenizer, DebertaV2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from matplotlib import pyplot as plt
import wandb
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score 
from gensim.models import KeyedVectors
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datasets import Dataset, load_metric
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

nltk.download('punkt')
nltk.download('stopwords')

Loading Datasets

Task 1

In [ ]:
task1_train_df = pd.read_csv('propaganda_train.tsv', delimiter='\t', quotechar='\'')
task1_train_df.loc[task1_train_df['label'] != "not_propaganda", 'label'] = "propaganda" # All 8 Propaganda labels are changed to "propaganda" and the total number of classes is reduced to 2
task1_train_df['tagged_in_context'] = task1_train_df['tagged_in_context'].str.replace('<BOS>', '', regex=False)
task1_train_df['tagged_in_context'] = task1_train_df['tagged_in_context'].str.replace('<EOS>', '', regex=False)

task1_test_df = pd.read_csv('propaganda_val.tsv', sep='\t', quotechar='\'') 
task1_test_df.loc[task1_test_df['label'] != "not_propaganda", 'label'] = "propaganda" # All 8 Propaganda labels are changed to "propaganda" and the total number of classes is reduced to 2
task1_test_df['tagged_in_context'] = task1_test_df['tagged_in_context'].str.replace('<BOS>', '', regex=False)
task1_test_df['tagged_in_context'] = task1_test_df['tagged_in_context'].str.replace('<EOS>', '', regex=False)

label_encoder = LabelEncoder()
task1_train_df['label'] = label_encoder.fit_transform(task1_train_df['label'])
task1_test_df['label'] = label_encoder.transform(task1_test_df['label'])

display(task1_train_df)
display(task1_test_df)

Task 2

In [ ]:
task2_train_df = pd.read_csv('propaganda_train.tsv', delimiter='\t', quotechar='\'')
task2_test_df = pd.read_csv('propaganda_val.tsv', delimiter='\t', quotechar='\'')

task2_train_df = task2_train_df[task2_train_df['label'] != 'not_propaganda']
task2_test_df = task2_test_df[task2_test_df['label'] != 'not_propaganda']

for index, row in task2_train_df.iterrows():
    bos_index = row['tagged_in_context'].index("<BOS>")
    eos_index = row['tagged_in_context'].index("<EOS>")
    task2_train_df.loc[index,'tagged_in_context'] = row['tagged_in_context'][bos_index + 5:eos_index]

for index, row in task2_test_df.iterrows():
    bos_index = row['tagged_in_context'].index("<BOS>")
    eos_index = row['tagged_in_context'].index("<EOS>")
    task2_test_df.loc[index,'tagged_in_context'] = row['tagged_in_context'][bos_index + 5:eos_index]

label_encoder = LabelEncoder()
task2_train_df['label'] = label_encoder.fit_transform(task2_train_df['label'])
task2_test_df['label'] = label_encoder.transform(task2_test_df['label'])


display(task2_train_df)
display(task2_test_df)

TASK-1 EDA

In [ ]:
# Plotting class distributions

class_counts_train = task1_train_df['label'].value_counts().sort_index()

class_counts_test = task1_test_df['label'].value_counts().sort_index()

df = pd.DataFrame({'Train': class_counts_train, 'Test': class_counts_test})

plt.figure(figsize=(12, 8))
df.plot(kind='barh', ax=plt.gca())
plt.title('Class Distribution in Training and Test Sets')
plt.xlabel('Frequency')
plt.ylabel('Classes')
plt.show()

# Plotting maximum, minimum and average word counts of the datasets

task1_train_df['word_count'] = task1_train_df['tagged_in_context'].apply(lambda x: len(x.split()))
task1_test_df['word_count'] = task1_test_df['tagged_in_context'].apply(lambda x: len(x.split()))

train_min = task1_train_df['word_count'].min()
train_max = task1_train_df['word_count'].max()
train_mean = task1_train_df['word_count'].mean()

test_min = task1_test_df['word_count'].min()
test_max = task1_test_df['word_count'].max()
test_mean = task1_test_df['word_count'].mean()

stats = pd.DataFrame({
    'Train': [train_min, train_max, train_mean],
    'Test': [test_min, test_max, test_mean]
}, index=['Minimum', 'Maximum', 'Average'])

stats.plot(kind='bar', figsize=(10, 6))
plt.title('Word Counts in Train and Test Sentences')
plt.ylabel('Word Count')
plt.xticks(rotation=0)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

TASK-2 EDA

In [ ]:
# Plotting class distributions

class_counts_train = task2_train_df['label'].value_counts().sort_index()

class_counts_test = task2_test_df['label'].value_counts().sort_index()

df = pd.DataFrame({'Train': class_counts_train, 'Test': class_counts_test})

plt.figure(figsize=(12, 8))
df.plot(kind='barh', ax=plt.gca())
plt.title('Class Distribution in Training and Test Sets')
plt.xlabel('Frequency')
plt.ylabel('Classes')
plt.show()

# Plotting maximum, minimum and average word counts of the datasets

task2_train_df['word_count'] = task2_train_df['tagged_in_context'].apply(lambda x: len(x.split()))
task2_test_df['word_count'] = task2_test_df['tagged_in_context'].apply(lambda x: len(x.split()))

train_min = task2_train_df['word_count'].min()
train_max = task2_train_df['word_count'].max()
train_mean = task2_train_df['word_count'].mean()

test_min = task2_test_df['word_count'].min()
test_max = task2_test_df['word_count'].max()
test_mean = task2_test_df['word_count'].mean()

stats = pd.DataFrame({
    'Train': [train_min, train_max, train_mean],
    'Test': [test_min, test_max, test_mean]
}, index=['Minimum', 'Maximum', 'Average'])

stats.plot(kind='bar', figsize=(10, 6))
plt.title('Word Counts in Train and Test Sentences')
plt.ylabel('Word Count')
plt.xticks(rotation=0)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

DeBERTaV3

In [ ]:
from datasets import Dataset
class DebertaV3():

    def __init__(self,train,test,n_labels):

        """
           Creates Dataset from train and test datasets. Creates model and tokenizer.
           :param train: Train set
           :param test: Test set
           :param n_labels: Number of classes
        """

        self.n_labels = n_labels     
        # Creating dataset, loading DeBERTaV3 model and tokenizer
        self.train_dataset = Dataset.from_pandas(train)
        self.test_dataset = Dataset.from_pandas(test)

        device = torch.device("mps")
        model_name = "microsoft/deberta-v3-base"
        self.tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)
        self.model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=n_labels)
        self.model.to(device)

    def tokenize(self, data):
        """
            It tokenizes the data given as parameters.
            :param data: Train or test data
            :return: Tokenized data 
        """
        return self.tokenizer(data['tagged_in_context'], padding="max_length", truncation=True, max_length=128)
    
    def create_datasets(self):
        """
            Tokenizes train and test sets
        """
        self.train_dataset = self.train_dataset.map(self.tokenize, batched=True)
        self.test_dataset = self.test_dataset.map(self.tokenize, batched=True)

    
    def evaluate(self, eval_pred):
        """
            Calculates Accuracy and F1 score. Creates confusion matrix.
            :return: Accuracy and F1 values with dictionary
        """
        accuracy_metric = load_metric("accuracy")
        f1_metric = load_metric("f1")

        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
        f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')

        cm = confusion_matrix(labels, predictions)
        np.save("confusion_matrix.npy", cm)
        
        return {"Accuracy" : accuracy, "F1" : f1}
    
    def train(self, b_size, epoch):

        """
            Trains and evaluates the model. Logs results to wandb.
            :param b_size: Batch size
            :param epoch: Number of epochs
        """

        #wandb.init(project="deberta-propaganda-detection-trainer") # Logging to wandb

        self.create_datasets()

        batch_size = b_size
        epochs = epoch
        torch.mps.empty_cache()
        log_time = int(self.train_dataset.num_rows / epochs)

        training_args = TrainingArguments(
            output_dir="./results",
            evaluation_strategy="epoch",
            learning_rate=2e-5,
            logging_strategy="epoch",
            logging_steps=log_time,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=epochs,
            weight_decay=0.01,
            use_mps_device=True,
            report_to="wandb"
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.test_dataset,
            compute_metrics=self.evaluate 
        )

        trainer.train()

        results = trainer.evaluate()
        print(results)

    def plot_confusion_matrix(self):

        """
            Plots confusion matrix and accuracy by class graph
        """

        # Loading confusion matrix saved by evaluate function
        cm = np.load("confusion_matrix.npy")
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] 
        
        df = pd.read_csv('propaganda_train.tsv', delimiter='\t')

        if self.n_labels == 2:
            df.loc[df['label'] != "not_propaganda", 'label'] = "propaganda"
        else:
            df = df[df['label'] != 'not_propaganda']

        labels = df['label'].unique()
        plt.figure(figsize=(8, 8))

        sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap='Blues')
        plt.xticks(range(1, self.n_labels + 1), labels=labels, rotation ='vertical')
        plt.yticks(range(1, self.n_labels + 1), labels=labels, rotation ='horizontal')
        plt.ylabel('True Labels')
        plt.xlabel('Predicted Labels')
        plt.title('Confusion Matrix')
        plt.margins(0.2) 
        plt.subplots_adjust(bottom = 0.15) 
        plt.show()


        # Calculating accuracy per class
        class_accuracy = 100 * cm_norm.diagonal() / cm_norm.sum(axis=1)

        plt.figure(figsize=(10, 8))
        sns.barplot(x=labels, y=class_accuracy, palette='viridis')
        plt.xlabel('Classes')
        plt.ylabel('Accuracy (%)')
        plt.title('Model Accuracy by Class')
        plt.ylim(0, 100)
        plt.xticks(rotation=90)
        plt.show()

# Run Task 2
# debertav3 = DebertaV3(task2_train_df, task2_test_df, len(task2_train_df["label"].unique()))
# debertav3.train(32, 1)
# debertav3.plot_confusion_matrix()

# Run Task 1
debertav3 = DebertaV3(task1_train_df, task1_test_df, len(task1_train_df["label"].unique()))
debertav3.train(32, 1)
debertav3.plot_confusion_matrix()

LSTM with BERT Word Embeddings

In [ ]:
from torch.utils.data import DataLoader, Dataset

# Creating LSTM with BERT Word Embeddings model
class LSTMWithBERTWordEmbeddings(nn.Module):
    def __init__(self, n_classes, bert_model):
        """
            Initializes model architecture
            :param n_classes: Number of classes
            :param bert_model: BERT model
        """
        super(LSTMWithBERTWordEmbeddings, self).__init__()
        self.bert = bert_model
        self.drop = nn.Dropout(p=0.3)
        self.lstm = nn.LSTM(self.bert.config.hidden_size, 128, num_layers=1, batch_first=True)
        self.out = nn.Linear(128, n_classes)

    def forward(self, input_ids, attention_mask):
        """
            Processes input through BERT and LSTM to generate class predictions.
            :param input_ids: Tensor of token IDs from tokenizer
            :param attention_mask: Tensor representing the attention mask
            :return: Output tensor from the final linear layer
        """
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        last_hidden_state = outputs['last_hidden_state']
        _, (hidden, _) = self.lstm(last_hidden_state)  # BERT output to LSTM
        hidden = self.drop(hidden.squeeze(0))
        return self.out(hidden)
    

class LSTMDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        """
           Initializes custom dataset.
           :param texts: Sentences
           :param labels: Labels
           :param tokenizer: Tokenizer
           :param max_len: Maximum sentence length
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        """
            Encodes and returns data from the given index
            :param idx: Index
            :return: dictionary containing input_ids, attention_mask and labels
        """
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class LBWE:

    def __init__(self, train, test, b_size, epoch, num_classes):

        """
           Initializes variables. Creates Dataset from train and test datasets. Creates model, tokenizer, optimizer and loss function. Creates dataloaders.
           :param train: Train set
           :param test: Test set
           :param b_size: Batch size
           :param epoch: Number of epoch
           :param num_classes: Number of classes
        """

        self.num_classes = num_classes

        # Loading BERT Model and Tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        bert_model = BertModel.from_pretrained('bert-base-uncased')

        train_dataset = LSTMDataset(train["tagged_in_context"], train["label"], self.tokenizer, max_len=128)
        test_dataset = LSTMDataset(test["tagged_in_context"], test["label"], self.tokenizer, max_len=128)

        # Creating dataloaders, defining model, optimizer and loss functions
        self.num_epochs = epoch

        self.train_loader = DataLoader(train_dataset, batch_size=b_size, shuffle=True)
        self.test_loader = DataLoader(test_dataset, batch_size=b_size)

        self.model = LSTMWithBERTWordEmbeddings(num_classes, bert_model)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=2e-5)
        self.criterion = nn.CrossEntropyLoss()

        self.device = torch.device("mps")
        self.model.to(self.device)
    

    def train(self):

        """
            Trains and evaluates the model. Logs results to wandb.
        """

        # Training
        for epoch in range(self.num_epochs):
            self.model.train()
            total_loss, total_accuracy = 0, 0

            for data in tqdm(self.train_loader):
                input_ids = data['input_ids'].to(self.device)
                attention_mask = data['attention_mask'].to(self.device)
                labels = data['labels'].to(self.device)

                outputs = self.model(input_ids, attention_mask)
                loss = self.criterion(outputs, labels)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()
                total_accuracy += (outputs.argmax(dim=1) == labels).sum().item()

            avg_train_loss = total_loss / len(self.train_loader)
            avg_train_accuracy = total_accuracy / len(self.train_loader.dataset)
            wandb.define_metric("epoch")
            wandb.define_metric("train_loss", step_metric="epoch")

            wandb.log({
                "train_loss": avg_train_loss,
                "epoch": epoch
                })
            
            wandb.define_metric("train_accuracy", step_metric="epoch")
            wandb.log({"train_accuracy": avg_train_accuracy,
                "epoch": epoch})
            print(f"Epoch {epoch+1}/{self.num_epochs}, Train Loss: {avg_train_loss:.5f}, Train Accuracy: {avg_train_accuracy:.5f}")

            # Evaluation
            self.model.eval()
            total_test_loss, total_test_accuracy = 0, 0
            predictions = []
            true_labels = []
            with torch.no_grad():
                for data in self.test_loader:
                    input_ids = data['input_ids'].to(self.device)
                    attention_mask = data['attention_mask'].to(self.device)
                    labels = data['labels'].to(self.device)

                    outputs = self.model(input_ids, attention_mask)
                    loss = self.criterion(outputs, labels)

                    total_test_loss += loss.item()
                    total_test_accuracy += (outputs.argmax(dim=1) == labels).sum().item()

                    _, preds = torch.max(outputs, dim=1)
                    predictions.extend(preds.tolist())
                    true_labels.extend(labels.tolist()) 

            avg_test_loss = total_test_loss / len(self.test_loader)
            avg_test_accuracy = total_test_accuracy / len(self.test_loader.dataset)
            f1 = f1_score(true_labels, predictions, average="macro") 
            wandb.define_metric("test_loss", step_metric="epoch")
            wandb.log({"test_loss": avg_test_loss,
                "epoch": epoch})
            wandb.define_metric("test_accuracy", step_metric="epoch")
            wandb.log({"test_accuracy": avg_test_accuracy,
                "epoch": epoch})
            wandb.define_metric("test_f1", step_metric="epoch")
            wandb.log({"test_f1": f1,
                "epoch": epoch})
            print(f"Epoch {epoch+1}/{self.num_epochs}, Test Loss: {avg_test_loss:.5f}, Test Accuracy: {avg_test_accuracy:.5f}")

    def plot_confusion_matrix(self):

        """
            Plots confusion matrix and accuracy by class graph
        """

        y_pred = []
        y_true = []

        self.model.eval()

        with torch.no_grad():
            for data in self.test_loader:
                input_ids = data['input_ids'].to(self.device)
                attention_mask = data['attention_mask'].to(self.device)
                labels = data['labels'].to(self.device)
                y_true.extend(labels.tolist()) 

                outputs = self.model(input_ids, attention_mask)
                loss = self.criterion(outputs, labels)

                _, preds = torch.max(outputs, dim=1)
                y_pred.extend(preds.tolist())


        df = pd.read_csv('propaganda_train.tsv', delimiter='\t')
        
        if self.num_classes == 2:
            df.loc[df['label'] != "not_propaganda", 'label'] = "propaganda"
        else:
            df = df[df['label'] != 'not_propaganda']
        labels = df['label'].unique()

        cm = confusion_matrix(y_true, y_pred)
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # Normalize the confusion matrix

        plt.figure(figsize=(8, 8))

        sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap='Blues')
        plt.xticks(range(1, self.num_classes + 1), labels=labels, rotation ='vertical')
        plt.yticks(range(1, self.num_classes + 1), labels=labels, rotation ='horizontal')
        plt.ylabel('True Labels')
        plt.xlabel('Predicted Labels')
        plt.title('Confusion Matrix')
        plt.margins(0.2) 
        plt.subplots_adjust(bottom = 0.15) 
        plt.show()

        class_accuracy = 100 * cm.diagonal() / cm.sum(axis=1)

        plt.figure(figsize=(10, 8))
        sns.barplot(x=labels, y=class_accuracy, palette='viridis')
        plt.xlabel('Classes')
        plt.ylabel('Accuracy (%)')
        plt.title('Model Accuracy by Class')
        plt.ylim(0, 100)
        plt.xticks(rotation=90)
        plt.show()

# Run Task 2
# lbwe = LBWE(task2_train_df, task2_test_df, 32, 1, len(task2_train_df["label"].unique()))
# lbwe.train()
# lbwe.plot_confusion_matrix()


# Run Task 1
lbwe = LBWE(task1_train_df, task1_test_df, 32, 1, len(task1_train_df["label"].unique()))
lbwe.train()
lbwe.plot_confusion_matrix()


SVM

In [ ]:
class SVM:

    def __init__(self,train,test):

        """
            Initializes train, test sets and word2vec model.
            :param train: Train set
            :param test: Test set
        """

        self.train_df = train
        self.test_df = test
        # Loading word2vec from local
        self.word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

    # Data preprocessing
    def preprocess_text(self,text):
        """
            Preprocesses text
            :param text: Sentence
            :return: Processed text
        """
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stop_words]
        return ' '.join(filtered_text)
    
    def normalise_dataframes(self):
        """
            Applies preprocessing to train and test dataframes
        """
        self.train_df['normalized_tagged_in_context'] = self.train_df['tagged_in_context'].apply(self.preprocess_text)
        self.test_df['normalized_tagged_in_context'] = self.test_df['tagged_in_context'].apply(self.preprocess_text)

    def create_sentence_vector(self,word2vec_model, sentence):
        """
            Creates sentence vectors with word2vec
            :param word2vec_model: Word2vec model
            :param sentence: Sentence
            :return: Sentence vector
        """
        words = sentence.split()
        word_vectors = [word2vec_model[word] for word in words if word in word2vec_model]
        if not word_vectors:
            return np.zeros(word2vec_model.vector_size)
        return np.mean(word_vectors, axis=0)
    
    def create_train_test(self):

        """
            Creates train and test sets for training
            :param word2vec_model: Word2vec model
            :param sentence: Sentence
            :return: Created train and test sets
        """

        self.normalise_dataframes()

        # Calculates sentence vector for all sentences in the dataframes
        self.train_df['doc_vector'] = self.train_df['normalized_tagged_in_context'].apply(lambda doc: self.create_sentence_vector(self.word2vec_model, doc))
        self.test_df['doc_vector'] = self.test_df['normalized_tagged_in_context'].apply(lambda doc: self.create_sentence_vector(self.word2vec_model, doc))

        # Creating x_train, y_train, x_test and y_test. Assigning vectors to x and assigning labels to y
        x_train = np.vstack(self.train_df['doc_vector'])
        y_train = self.train_df['label'].values
        x_test = np.vstack(self.test_df['doc_vector'])
        y_test = self.test_df['label'].values

        return x_train,y_train,x_test,y_test
    

    def train(self,x_train,y_train,x_test,y_test):

        """
            Trains the model
            :param x_train: Training sentences
            :param y_train: Training labels
            :param x_tets: Test sentences
            :param y_test: Test labels
            :return: GridSearchCV results
        """


        #Running GridSearchCV
        param_grid = {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
            'gamma': ['scale', 'auto', 0.1, 1, 10]
        }

        grid_search = GridSearchCV(SVC(), param_grid, refit=True, verbose=3, scoring='f1_macro', cv=2)
        grid_search.fit(x_train, y_train)
        results = pd.DataFrame(grid_search.cv_results_)

        return results

    def plot_results(self, results):

        """
            Plots results with hetmap for every kernel
            :param results: GridSearchCV results
        """

        # Creating subplots for every kernel
        kernels = results['param_kernel'].unique()
        num_kernels = len(kernels)
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))
        axes = axes.flatten()

        for idx, kernel in enumerate(kernels):
            kernel_results = results[results['param_kernel'] == kernel]
            
            pivot_table = kernel_results.pivot_table(values='mean_test_score', index='param_C', columns='param_gamma', aggfunc=np.max)
            
            sns.heatmap(pivot_table, ax=axes[idx], annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'F1 Score'})
            axes[idx].set_title(f'Task - 1 SVM F1 Score Optimization for Kernel: {kernel}')
            axes[idx].set_xlabel('Gamma')
            axes[idx].set_ylabel('C')

        for ax in axes[num_kernels:]:
            fig.delaxes(ax)

        fig.tight_layout(pad=3.0)
        plt.show()

    def plot_confusion_matrix(self,x_train, y_train, kernel_param, c_param, gamma_param): 

        """
            Plots confusion matrix
            :param x_train: Training sentences
            :param y_train: Training labels
            :param kernel_param: Kernel type
            :param c_param: C value
            :param gamma_param: Gamma value
        """   

        # Train with best hyperparameters
        svm_classifier = SVC(kernel=kernel_param, C=c_param, gamma=gamma_param)
        svm_classifier.fit(x_train, y_train)

        # Predict
        y_pred = svm_classifier.predict(x_test)

        print("Accuracy:", accuracy_score(y_test, y_pred))
        cm_normalized = confusion_matrix(y_test, y_pred, normalize='true')

        df = pd.read_csv('propaganda_train.tsv', delimiter='\t')
        df.loc[df['label'] != "not_propaganda", 'label'] = "propaganda"
        labels = df['label'].unique()

        plt.figure(figsize=(8, 8))
        sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap='Blues')
        plt.xticks(range(1,len(labels) + 1), labels=labels, rotation ='vertical')
        plt.yticks(range(1,len(labels) + 1), labels=labels, rotation ='horizontal')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.show()



# Run Task 1
# svm = SVM(task1_train_df,task1_test_df)
# x_train,y_train,x_test,y_test = svm.create_train_test()
# results = svm.train(x_train,y_train,x_test,y_test)
# svm.plot_results(results)
# svm.plot_confusion_matrix(x_train,y_train,"rbf",1,1)

# Run Task 2
svm = SVM(task2_train_df,task2_test_df)
x_train,y_train,x_test,y_test = svm.create_train_test()
results = svm.train(x_train,y_train,x_test,y_test)
svm.plot_results(results)
svm.plot_confusion_matrix(x_train,y_train,"rbf",1,1)

